<a href="https://colab.research.google.com/github/RajSinha77/DL-Reg-implementation-/blob/main/DLReg_vs_L2_on_Oxford_IIIT_Pet_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


In [4]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_dataset

# Define constants
BATCH_SIZE = 64
NUM_EPOCHS = 25
LEARNING_RATE = 0.001

# Load the Oxford-IIIT Pet dataset from Hugging Face
dataset = load_dataset('timm/oxford-iiit-pet')

# Print the dataset structure to check available splits
print(dataset)

# Identify the correct splits
train_split = 'train'
val_split = 'test'

# Custom dataset class to handle Hugging Face dataset
class OxfordIIITPetDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = sample['image']
        label = sample['label']
        if self.transform:
            image = self.transform(image)
        return image, label

# Transformations for the Oxford-IIIT Pet dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.convert("RGB") if x.mode != "RGB" else x),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Prepare the dataset and dataloaders
train_dataset = OxfordIIITPetDataset(dataset[train_split], transform=transform)
val_dataset = OxfordIIITPetDataset(dataset[val_split], transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Define the models with updated weights parameter
resnet152 = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)
densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)

# Modify the final layers to fit Oxford-IIIT Pet classes (37 classes)
num_ftrs_resnet = resnet152.fc.in_features
resnet152.fc = nn.Linear(num_ftrs_resnet, 37)

num_ftrs_densenet = densenet.classifier.in_features
densenet.classifier = nn.Linear(num_ftrs_densenet, 37)

# Define DL-Regularization
class DLRegularization(nn.Module):
    def __init__(self, model, lambda_reg=0.001):
        super(DLRegularization, self).__init__()
        self.model = model
        self.lambda_reg = lambda_reg

    def forward(self, x):
        output = self.model(x)
        return output

    def dl_reg_loss(self, x, output):
        Z = nn.Linear(x.size(1), output.size(1), bias=False).to(x.device)
        reg_loss = torch.norm(Z(x) - output) ** 2
        return self.lambda_reg * reg_loss

# Training function
def train_model(model, criterion, optimizer, num_epochs=NUM_EPOCHS, dl_regularizer=None):
    model.train()
    train_acc = []
    val_acc = []

    for epoch in range(num_epochs):
        running_corrects = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            if dl_regularizer:
                reg_loss = dl_regularizer.dl_reg_loss(inputs, outputs)
                loss += reg_loss

            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

        epoch_acc = running_corrects.double() / len(train_dataset)
        train_acc.append(epoch_acc.item())

        # Validation
        model.eval()
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        epoch_val_acc = val_corrects.double() / len(val_dataset)
        val_acc.append(epoch_val_acc.item())

        print(f'Epoch {epoch+1}/{num_epochs}, Train Acc: {epoch_acc.item()}, Val Acc: {epoch_val_acc.item()}')

    return train_acc, val_acc

# Initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Experiment with L2 regularization
resnet152 = resnet152.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet152.parameters(), lr=LEARNING_RATE, weight_decay=0.001)
train_acc_l2, val_acc_l2 = train_model(resnet152, criterion, optimizer)

# Experiment with DL-Regularization
densenet = densenet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(densenet.parameters(), lr=LEARNING_RATE)
dl_regularizer = DLRegularization(densenet, lambda_reg=0.001)
train_acc_dl, val_acc_dl = train_model(densenet, criterion, optimizer, dl_regularizer=dl_regularizer)

# Plotting the results
epochs = range(1, NUM_EPOCHS + 1)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_acc_l2, label='L2 Regularization')
plt.plot(epochs, train_acc_dl, label='DL-Regularization')
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, val_acc_l2, label='L2 Regularization')
plt.plot(epochs, val_acc_dl, label='DL-Regularization')
plt.title('Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


DatasetDict({
    train: Dataset({
        features: ['image', 'label', 'image_id', 'label_cat_dog'],
        num_rows: 3680
    })
    test: Dataset({
        features: ['image', 'label', 'image_id', 'label_cat_dog'],
        num_rows: 3669
    })
})
Epoch 1/25, Train Acc: 0.06304347826086956, Val Acc: 0.12319433088034887
Epoch 2/25, Train Acc: 0.4125, Val Acc: 0.6617606977378032
Epoch 3/25, Train Acc: 0.8548913043478261, Val Acc: 0.8571817934041973
Epoch 4/25, Train Acc: 0.9279891304347826, Val Acc: 0.8904333605887162
Epoch 5/25, Train Acc: 0.9554347826086956, Val Acc: 0.8956118833469611
Epoch 6/25, Train Acc: 0.9671195652173913, Val Acc: 0.8571817934041973
Epoch 7/25, Train Acc: 0.9815217391304348, Val Acc: 0.8958844371763424
Epoch 8/25, Train Acc: 0.991304347826087, Val Acc: 0.8958844371763424
Epoch 9/25, Train Acc: 0.9921195652173913, Val Acc: 0.9100572363041701
Epoch 10/25, Train Acc: 0.9983695652173913, Val Acc: 0.9086944671572635
Epoch 11/25, Train Acc: 0.9991847826086957, Val 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (43008x224 and 3x37)